# FairSeq NMT Tutorial
**F**acebook **AI R**esearch **Seq**uence-to-Sequence Toolkit written in Python

A Fast, Extensible Toolkit for Sequence Modeling

Reference
- https://fairseq.readthedocs.io/en/latest/command_line_tools.html
- https://rlqof7ogm.toastcdn.net/references/2021_session_20.pdf
- https://github.com/matsunagadaiki151/FairseqTutorial/blob/main/FairseqTranslation.ipynb
- https://github.com/Lainshower/jeju-blues/blob/main/Ko-Je%20translation/transformer_translation.ipynb

## Mount Drive & Files

In [17]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
my_path = '/content/notebooks'
os.symlink('/content/drive/MyDrive/AllforOne/package_collection', my_path)
sys.path.insert(0, my_path)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-f3163e8b368f>", line 7, in <module>
    os.symlink('/content/drive/MyDrive/AllforOne/package_collection', my_path)
FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/AllforOne/package_collection' -> '/content/notebooks'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileExistsError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most 

FileExistsError: ignored

In [ ]:
!nvidia-smi

Mon Mar 27 04:26:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 1. Dataset
AI Hub 'Korean-English pair corpus'  
https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=126

Same as OpenNMT Tutorial.  
https://github.com/Judy-Choi/NMT_Series/blob/main/Model/OpenNMT/NMT_ko-en.ipynb

You can download all files from :  
https://drive.google.com/drive/folders/1xgNQaaEqJArx3iofoC4JJ8tRmfPSZTTM?usp=share_link


### Split Dataset
- Val(Dev) : 5,000
- Test : 3,000
- Train : 1,493,750

## 2. Subword Tokenization
Same as OpenNMT Tutorial.
- https://github.com/Judy-Choi/NMT_Series/blob/main/Model/OpenNMT/NMT_ko-en.ipynb

SentencePiece
- Unsupervised text tokenizer and detokenize
- Not depend on language
- Not depend on Spacing or not
- Alleviate the open vocabulary problems (OOV)
- Supports **BPE(Byte-Pair-Encoding), Unigram** language model

## 3. Model
### FairSeq
https://github.com/facebookresearch/fairseq

Fairseq(-py) is a sequence modeling toolkit that allows researchers and developers to train custom models for :
- translation
- summarization
- language modeling
- other text generation tasks

This toolkit supports
- Distributed training
across multiple GPUs and machines.
- Fast mixed-precision training and inference on modern GPUs
- Pytorch

### Install Fairseq

In [ ]:
!git clone https://github.com/pytorch/fairseq

Cloning into 'fairseq'...
remote: Enumerating objects: 34534, done.
remote: Total 34534 (delta 0), reused 0 (delta 0), pack-reused 34534
Receiving objects: 100% (34534/34534), 24.06 MiB | 19.98 MiB/s, done.
Resolving deltas: 100% (25109/25109), done.


In [ ]:
cd /content/fairseq

/content/fairseq


In [ ]:
ls

CODE_OF_CONDUCT.md  fairseq/        LICENSE         RELEASE.md        setup.py
CONTRIBUTING.md     fairseq_cli/    MANIFEST.in     release_utils.py  tests/
docs/               hubconf.py      pyproject.toml  scripts/          train.py
examples/           hydra_plugins/  README.md       setup.cfg


In [ ]:
!pip install --editable ./

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.6/269.6 KB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 KB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Building editable for fairseq (pyproject.toml) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-0.editable-cp39-cp39-linux_x86_64.whl size=9192 sha256=6d6e3178c1938a14b9ad5f42f49082eec84eaa8d5c02b086e3febe23313cead6
  Stored in directory: /tmp/pip-ephem-wheel-cache-in9_cbhb/whee

### Install Libraries

In [ ]:
!pip install pyproject-toml
# For large datasets
!pip install pyarrow
# For tensorboard log
!pip install tensorboardX
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 10.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 34.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=46d99920236c1af8ef00d0ca70daebe59a75a1d19b31ffcddebac7c4575b4861
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses


### Command-line Tools
https://fairseq.readthedocs.io/en/latest/command_line_tools.html

Fairseq provides several command-line tools for training and evaluating models:

- fairseq-preprocess: Data pre-processing: build vocabularies and binarize training data
- fairseq-train: Train a new model on one or multiple GPUs
- fairseq-generate: Translate pre-processed data with a trained model
- fairseq-interactive: Translate raw text with a trained model
- fairseq-score: BLEU scoring of generated translations against reference translations
- fairseq-eval-lm: Language model evaluation

### Preprocess

In [ ]:
cd /content/drive/MyDrive/AllforOne/Lecture/Fairseq

/content/drive/MyDrive/AllforOne/Lecture/Fairseq


In [ ]:
dir = "/content/drive/MyDrive/AllforOne/Lecture/Fairseq"

In [ ]:
!fairseq-preprocess \
    --source-lang ko \
    --target-lang en \
    --validpref $dir/dataset/valid \
    --trainpref $dir/dataset/train \
    --testpref $dir/dataset/test \
    # --thresholdtgt 3 \
    # --thresholdsrc 3 \
    # --workers 8 \
    --destdir $dir/preprocess

2023-03-24 15:52:31 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, aim_repo=None, aim_run_hash=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, quantization_config_path=None, profile=False, reset_logging=False, suppress_crashes=False, use_plasma_view=False, plasma_path='/tmp/plasma', criterion='cross_entropy', tokenizer=None, bpe=None, optimizer=None, lr_scheduler='fixed', scoring='bleu', task='translation', source_lang='ko', target_lang='en', tr

### Train

#### fairseq-train arguments
Example

```
fairseq-train \
    [preprocess_dir] \
    --source-lang ko \
    --target-lang en \
    --arch transformer \
    --max-sentences 50 \
    --optimizer adam
    --save-dir [checkpoint_dir \
```

**Model**  
- --arch [model]   
- --optimizer [optimizer]   
- --max-epoch [force stop training at specified epoch]   
- --batch-size, --max-sentences [number of examples in a batch]   

**Set validation metric (ex: BLEU)**   
- --scoring [scoreing metric]   
- --best-checkpoint-metric [metric to use for saving “best” checkpoints]   

(You should add these arguments too.  
Only '--best-checkpoint-metric' bleu flag doesn't work alone)
- --eval-bleu
- --eval-bleu-args
- --eval-bleu-detok moses
- --eval-bleu-remove-bpe
- --eval-bleu-print-samples

**Save checkpoint**
- --maximize-best-checkpoint-metric
  - select the largest metric value for saving “best” checkpoints
- --no-epoch-checkpoints
  - only store last and best checkpoints
- --save-dir [path to save checkpoints]

In [ ]:
cd /content/drive/MyDrive/AllforOne/Lecture/Fairseq

/content/drive/MyDrive/AllforOne/Lecture/Fairseq


In [ ]:
!fairseq-train /content/drive/MyDrive/AllforOne/Lecture/Fairseq/preprocess \
    --source-lang ko \
    --target-lang en \
    --task translation \
    --arch transformer \
    --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' \
    --clip-norm 0.0 \
    --lr 0.0005 \
    --lr-scheduler inverse_sqrt \
    --label-smoothing 0.1 \
    --dropout 0.3 \
    --max-tokens 4000 \
    --stop-min-lr '1e-09' \
    --lr-scheduler inverse_sqrt       \
    --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy       \
    --warmup-updates 4000 \
    --warmup-init-lr '1e-07'    \
    --skip-invalid-size-inputs-valid-test \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --max-epoch 100 \
    --no-epoch-checkpoints \
    --save-dir "/content/drive/MyDrive/AllforOne/Lecture/Fairseq/checkpoints"

2023-03-27 09:16:28 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': N

In [ ]:
!fairseq-generate /content/drive/MyDrive/AllforOne/Lecture/Fairseq/preprocess/ \
--path /content/drive/MyDrive/AllforOne/Lecture/Fairseq/checkpoints/checkpoint_best.pt \
--source-lang ko --target-lang en \
--max-sentences 100 \
--beam 5 \
--task translation \
--gen-subset test \
--skip-invalid-size-inputs-valid-test \
--batch-size 64 \
--remove-bpe sentencepiece \
--results-path prediction

2023-03-27 11:04:19 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name'

In [ ]:
!grep '^H' prediction/generate-test.txt | cut -f3- > prediction/gen.out.sys # 예측된 문장 (H)
!grep '^T' prediction/generate-test.txt | cut -f2- > prediction/gen.out.ref # 타겟(정답) 문장 (T)

In [ ]:
!fairseq-score \
--sys prediction/gen.out.sys \
--ref prediction/gen.out.ref

Namespace(sys='prediction/gen.out.sys', ref='prediction/gen.out.ref', order=4, ignore_case=False, sacrebleu=False, sentence_bleu=False)
BLEU4 = 5.42, 25.4/7.4/3.0/1.5 (BP=1.000, ratio=1.051, syslen=74620, reflen=70990)


In [18]:
# Generate test result (BLEU)
!tail -n 1 /content/drive/MyDrive/AllforOne/Lecture/Fairseq/prediction/generate-test.txt

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
Generate test with beam=5: BLEU4 = 28.36, 56.6/34.3/22.6/15.6 (BP=0.985, ratio=0.985, syslen=69947, reflen=70990)
